# Задание 1. SQL

In [1]:
import pandas as pd
import duckdb

In [2]:
users = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/users.csv')
course_users = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/course_users.csv')
courses = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/courses.csv')
course_types = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/course_types.csv')
lessons = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/lessons.csv')
subjects = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/subjects.csv')
cities = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/cities.csv')
homework_done = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/homework_done.csv')
homework = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/homework.csv')
homework_lessons = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/homework_lessons.csv')
user_roles = pd.read_csv('/Users/dmitriy/Downloads/Тестовое задание/Файлы/user_roles.csv') 

In [3]:
query = """
SELECT c.id,
    c.name AS course_name,
    s.name AS subject_name,
    s.project AS type,
    ct.name AS course_type,
    c.starts_at,
    cu.user_id AS user_id,
    u.last_name,
    cty.name AS city,
    cu.active,
    CAST(IF(cu.created_at < c.starts_at, c.starts_at, cu.created_at) AS DATE) AS course_opened,
    CAST(FLOOR(('2024-11-12' - course_opened)/31)AS int) AS comp_months
FROM users AS u
JOIN course_users AS cu ON u.id = cu.user_id
JOIN courses AS c ON cu.course_id = c.id
JOIN subjects AS s ON c.subject_id = s.id
JOIN course_types AS ct ON c.course_type_id = ct.id
JOIN cities AS cty ON u.city_id = cty.id
WHERE ct.name == 'Годовой' 
GROUP BY c.id, c.name, c.starts_at, cu.user_id, u.last_name, cty.name, s.name, s. project, ct.name, course_opened, cu.active;
"""
df_result = duckdb.query(query).to_df()
display(df_result)

,id,course_name,subject_name,type,course_type,starts_at,user_id,last_name,city,active,course_opened,comp_months
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10009,Чернобель,Минеральные Воды,1,2024-09-16,1
1,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10020,Бархатова,Москва,1,2024-09-16,1
2,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10022,Ролдугина,Донецк,1,2024-10-12,1
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10029,Болотин,Санкт-Петербург,1,2024-09-16,1
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10052,Глухова,Ессентуки,1,2024-09-16,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5380,22,Годовой 2к25 стандарт,Литература,ЕГЭ,Годовой,2024-09-17,33854,Сайгушева,Ульяновск,1,2024-10-11,1
5381,17,Годовой 2к25 стандарт,Английский язык,ЕГЭ,Годовой,2024-09-20,33969,Мелекесова,Салават,1,2024-10-11,1
5382,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,33197,Самохина,Энгельс,1,2024-10-11,1
5383,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,33232,Fist,Ярославль,1,2024-10-10,1


In [4]:
query = """
SELECT u.id AS user_id,
       c.id AS id,
       COUNT(hd.id) AS cnt
FROM users AS u 
JOIN homework_done AS hd ON u.id = hd.user_id
JOIN homework_lessons AS hl ON hd.homework_id = hl.homework_id
JOIN lessons AS l ON hl.lesson_id = l.id
JOIN courses AS c ON l.course_id = c.id
WHERE u.id == '10010'
GROUP BY u.id, c.id
"""
df_result = duckdb.query(query).to_df()
display(df_result)

,user_id,id,cnt
0,10010,56,8
1,10010,19,12
2,10010,59,1
3,10010,58,5
4,10010,18,15
5,10010,79,7
6,10010,16,15
7,10010,81,7


In [6]:
query = """
-- Задание 1. SQL
-- Создаю табличное выражение с двумя таблицами, 1 - все поля, кроме последнего задания, 2 - последнее задание
WITH tab_1 AS
    (SELECT c.id, -- ID курса
        c.name AS course_name, -- Название курса
        s.name AS subject_name, -- Предмет
        s.project AS type, -- Тип предмета (ЕГЭ и ОГЭ)
        ct.name AS course_type, -- Тип курса (нужен только годовой)
        c.starts_at, -- Дата старта курса
        cu.user_id AS user_id, -- ID ученика
        u.last_name, -- Фамилия ученика
        cty.name AS city, -- Город ученика
        cu.active, -- Ученик не отчислен с курса
        CAST(IF(cu.created_at < c.starts_at, c.starts_at, cu.created_at) AS DATE) AS course_opened, -- Дата открытия курса ученику
        /* Пошел по такой логике: если ученик зарегистрировался на курс (course_users.created_at) раньше, 
        чем он начался (courses.starts_at), то датой открытия считаю день старта курса. Если же позже,
        чем курс начался, то датой открытия считаю дату регистрации на курс */
        CAST(FLOOR(('2024-11-12' - course_opened)/31)AS int) AS comp_months -- Сколько полных месяцев курса открыто у ученика
        /* Здесь же - считаю от даты открытия курса до сегодняшней даты (11 ноября 2024). Возможно, 
        курс открывает все темы, если ученик зарегистрировался на курс позже, чем он начался. В таком случае,
        количество полных месяцев нужно считать от даты старта курса (courses.starts_at) */
    FROM users AS u -- таблица с пользователями
    JOIN course_users AS cu ON u.id = cu.user_id -- джоин связки курсов и пользователей
    JOIN courses AS c ON cu.course_id = c.id -- джоин курсов
    JOIN subjects AS s ON c.subject_id = s.id -- джоин предметов
    JOIN course_types AS ct ON c.course_type_id = ct.id -- джоин типов курсов
    JOIN cities AS cty ON u.city_id = cty.id -- джоин таблицы с городами
    WHERE ct.name == 'Годовой' -- фильтрация учеников с годовых курсов. ЕГЭ и ОГЭ не фильтровал, так как в типе предмета есть только два этих значения
    GROUP BY c.id, c.name, c.starts_at, cu.user_id, u.last_name, cty.name, s.name, s. project, ct.name, course_opened, cu.active), -- группировка
    
    tab_2 AS -- вторая таблица с количеством сданных ДЗ ученика на определенном курсе
        (SELECT u.id AS user_id, -- ID ученика
               c.id AS id, -- ID курса
               COUNT(hd.id) AS cnt -- количество сданных ДЗ (некоторые могут быть сданы, но не оценены, поэтому по id)
    FROM users AS u -- пользователи
    JOIN homework_done AS hd ON u.id = hd.user_id -- джоин сданных домашних работы
    JOIN homework_lessons AS hl ON hd.homework_id = hl.homework_id -- джоин связки 
    JOIN lessons AS l ON hl.lesson_id = l.id -- джоин уроков
    JOIN courses AS c ON l.course_id = c.id -- джоин курсов, чтобы связать количество домашки по ID ученика и курса
    GROUP BY u.id, c.id) -- группировка
     
SELECT tab_1.*, -- с таблицы 1 берем все
        tab_2.cnt -- с таблицы 2 берем только столбик с числом сданных ДЗ
FROM tab_1 -- с таблицы 1
JOIN tab_2 ON tab_1.id = tab_2.id AND tab_1.user_id = tab_2.user_id -- джоиним таблицу 2 по столбцу с ID пользователя и ID курса
"""
df_result = duckdb.query(query).to_df()
display(df_result)

,id,course_name,subject_name,type,course_type,starts_at,user_id,last_name,city,active,course_opened,comp_months,cnt
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10009,Чернобель,Минеральные Воды,1,2024-09-16,1,16
1,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10014,Павлова,Краснодар,0,2024-09-16,1,4
2,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10017,Парфёнова,Хабаровск,1,2024-09-17,1,6
3,22,Годовой 2к25 стандарт,Литература,ЕГЭ,Годовой,2024-09-17,10043,Киреева,Брянск,1,2024-09-17,1,12
4,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10046,Шишман,Ростов-на-Дону,1,2024-09-21,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10313,Kiriianova,Сочи,1,2024-09-16,1,4
4877,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,30237,Буйлова,Пермь,1,2024-10-01,1,8
4878,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,30504,Евдокимова,Москва,1,2024-10-05,1,14
4879,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,30716,Фарафонова,Сургут,1,2024-10-03,1,14
